In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License")
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an \"AS IS\" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# TensorFlow Hub is a repository of models
# We'll pick one from there to get the already-learned features
import tensorflow_hub as hub

In [ ]:
# This uses TFDS to load the beans dataset
# I covered using TFDS in my book 'AI and Machine Learning for Programmers'
(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    name = 'beans', 
    split = ['train', 'validation', 'test'],
    as_supervised = True,
    with_info = True)

In [ ]:
# This continues setting up the data from TFDS to use in training the model
batch_size=32
def map_data(image, label, target_height = 224, target_width = 224):
    """Normalizes images: `unit8` -> `float32` and resizes images
    by keeping the aspect ratio the same without distortion."""
    image = tf.cast(image, tf.float32)/255.
    image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)
    return image, label

ds_train = ds_train.map(map_data)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_validation = ds_validation.map(map_data)
ds_validation = ds_validation.batch(batch_size)
ds_validation = ds_validation.cache()
ds_validation = ds_validation.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(map_data)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# You can experiment with different model types
# The commented out one is a good model for crop diseases, as it was already trained on Cassava blight
# The second one is MobileNet, a common one for mobile applications
#model_handle = "https://tfhub.dev/google/cropnet/feature_vector/cassava_disease_V1/1"
model_handle = "https://tfhub.dev/google/imagenet/mobilenet_v2_035_224/feature_vector/4"

In [ ]:
# HEre is where we take the model from hub, treat it as a layer called 'Feature Vector'
# and add our own model beneath
feature_vector = hub.KerasLayer(model_handle, trainable=False,
                               input_shape=(224, 224, 3))

model = tf.keras.models.Sequential([
  feature_vector,
  tf.keras.layers.Dense(3, activation = 'softmax'),
])

In [ ]:
model.summary()

In [ ]:
# Here is where we define the parameters to use when training the model
# The loss function and the optimizer control how it learns
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(.001),
    metrics = ['accuracy'],
)

In [ ]:
# We can then train the model. 20 epochs with GPU in colab takes less than a minute!
num_epochs = 20
history = model.fit(
    ds_train,
    epochs = num_epochs,
    validation_data = ds_validation,
    verbose=1
)

In [ ]:
# We can save the model to later convert it to JS or TFLite
tf.saved_model.save(model, '/tmp/saved_model/')

In [ ]:
# Here we can plot the loss and accuracy of the model as it trained
# Accuracy should go up over time, loss should go down
# THere are 2 curves -- for the training data, which the model used to 'figure out' how to fit images to labels
# And for validation data, which wasn't used in the fitting, but can be a nice measurement of how accurate the model is
# on data that it hadn't previously seen. In a good model, these curves will end up very close to each other
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(num_epochs)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# This model shows that the accuracy begins to diverge at about 15 epochs
# which is a sign of overfitting, so it's not a great model.
# But it's good enough for now! 

In [ ]:
# That overfitting is borne out in the final test accuracy, 
# which is about 89%, where the model accuracy was about 92%
# This shows that the model does better on data that it has 'seen' while
# training, but not so well on data it hasn't yet seen. 

test_loss, test_acc = model.evaluate(ds_test)
print('n Final test accuracy:', test_acc)

In [ ]:
def return_class_labels(ds):
    """"Returns a list of class labels from a `DatasetV1Adapter` object."""
    l_labels = []
    for _, labels in ds.take(-1):
        labels = labels.numpy()
        l_labels.append(labels[:])
    return [item for sublist in l_labels for item in sublist]

def get_text_label(labelval):
  labels = {
      0: "Angular Leaf Spot",
      1: "Leaf Rust",
      2: "Healthy"
  }
  return labels.get(labelval)

In [ ]:
# This code will plot out a bunch of images, telling us the actual label (the diagnosed disease)
# and the predicted label (what the model thinks the disease is)
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
normalized_probs = probability_model.predict(ds_test)
predicted_labels = np.argmax(normalized_probs, axis = 1)
actual_labels = return_class_labels(ds_test)

# Looking at test images
example = ds_test.take(1)
for sample in example:
    image = sample[0]
    image = image.numpy()

n_cols, n_rows = 4, 4
plt.rcParams['figure.figsize'] = [n_cols*8, n_rows*8]

fig = plt.figure()
for i in range(1, n_cols*n_rows + 1):
    ax = fig.add_subplot(n_rows, n_cols,i)
    ax.text(5, -9, "actual: " + get_text_label(actual_labels[i]) + ", predicted: " + get_text_label(predicted_labels[i]) ,
            color = 'red', fontsize = 15)
    ax.imshow(image[i, :, :, :], cmap = plt.get_cmap("jet"))